# Euler topology and band node braiding

Here we show how to build a $6 \times 6$ Bloch Hamiltonian including s and p bands of the honeycomb lattice. We then compute the Euler curvature and patch euler characteristics using functions from the 'topology' submodules, before finally demonstrating the transfer of quaternion charges between gaps of the p bands.

## Building the latttice

In [17]:
from tightbinding.geometry import Orbital, Site, Unitcell, Lattice

s = Orbital('s')
px = Orbital('px')
py = Orbital('py')

a = 3.36 # Lattice constant used in the main article describing this work
a_site = Site('A', [0, a/2], [s, px, py])
b_site = Site('B', [0, -a/2], [s, px, py])

a1 = [-3**0.5/2 * a, 3/2 * a] # 1st lattice vector
a2 = [3**0.5/2 * a, 3/2 * a] # 2nd lattice vector
uc = Unitcell("honeycomb", [a1,a2])
uc.add_site([a_site,b_site])

# Creating a periodic lattice made of a single unit cell
honeycomb = Lattice('honeycomb', uc, (True, True))
honeycomb.add_unitcell((0,0), update=True)
honeycomb.plot()


## Building the Hamiltonian

Here we consider only nearest-neighbour couplings, and neglect transverse couplings for the p-bands. We also take into account a strain ($\beta$) of the lattice along the y-axis, and ellipticity of the pillars ($\epsilon_{\text{el.}}$ and $\theta_{\text{el.}}$). This ellipticity term introduce an on-site energy difference and a intra-site coupling (in the $p_x,p_y$ basis). It is pure energy difference when $\theta_{\text{el.}} = 0 \: [\frac{\pi}{2}]$ and pure coupling when $\theta_{\text{el.}} = \frac{\pi}{4} \: [\frac{\pi}{2}]$

In [18]:
from tightbinding.hamiltonian import Energy, Hopping, Hamiltonianbuilder
import numpy as np

kl = 1.2
n_k = 51

parameters = {
    "kx": np.linspace(-kl, kl, n_k),
    "ky": np.linspace(-kl, kl, n_k),
    "delta_e": 5, # The energy diffrence between s and p orbitals
    "ts": -0.2, # hopping strength between s-orbitals
    "tsp": -0.1, # s-p hopping strength
    "tp": 1, # p-orbital hopping strength
    "strain": np.linspace(1., 1.5, 5),
    "ellipticity": np.linspace(0, 3.3, 5),
    "theta_ell": np.linspace(0,np.pi/4, 2), # angle between the ellipse axis and x-axis, in radians
}

onsite = Energy(honeycomb)
onsite.set_energy("-delta_e", norbital='s') # shifting the s-orbitals down
onsite.set_energy("np.cos(2*theta_ell)*ellipticity/2", norbital="px") # ellipticity term for px
onsite.set_energy("-np.cos(2*theta_ell)*ellipticity/2", norbital="py") # ellipticity term for py

links = [(0,0), (1,0), (0,1)]
links_strain = ["strain", "1", "1"]
links_angle = [-np.pi/2 + 2*np.pi/3*k for k in range(3)] # The links direction, important to compute the hopping strength between p orbitals

couplings = [Hopping(honeycomb, f"ts * {strain}", "A_s", "B_s", link) for link, strain in zip(links, links_strain)] # s-s coupling

couplings += [Hopping(honeycomb, f"np.cos({angle}) * np.cos({angle}) * tp * {strain}", "A_px", "B_px", link) for angle, link, strain in zip(links_angle, links, links_strain)] # px-px hoppings
couplings += [Hopping(honeycomb, f"np.cos({angle}) * np.sin({angle}) * tp * {strain}", "A_px", "B_py", link) for angle, link, strain in zip(links_angle, links, links_strain)] # px-py hoppings
couplings += [Hopping(honeycomb, f"np.sin({angle}) * np.cos({angle}) * tp * {strain}", "A_py", "B_px", link) for angle, link, strain in zip(links_angle, links, links_strain)] # py-px hoppings
couplings += [Hopping(honeycomb, f"np.sin({angle}) * np.sin({angle}) * tp * {strain}", "A_py", "B_py", link) for angle, link, strain in zip(links_angle, links, links_strain)] # py-py hoppings

couplings += [Hopping(honeycomb, f"np.cos({angle}) * tsp * {strain}", "A_s", "B_px", link) for angle, link, strain in zip(links_angle, links, links_strain)] # s-px hoppings
couplings += [Hopping(honeycomb, f"np.sin({angle}) * tsp * {strain}", "A_s", "B_py", link) for angle, link, strain in zip(links_angle, links, links_strain)] # s-py hoppings
couplings += [Hopping(honeycomb, f"np.cos({-angle}) * tsp * {strain}", "A_px", "B_s", link) for angle, link, strain in zip(links_angle, links, links_strain)] # px-s hoppings
couplings += [Hopping(honeycomb, f"np.sin({-angle}) * tsp * {strain}", "A_py", "B_s", link) for angle, link, strain in zip(links_angle, links, links_strain)] # py-s hoppings

couplings += [Hopping(honeycomb, "np.sin(2 * theta_ell) * ellipticity / 2", "A_px", "A_py", (0,0))] # intra-site ellipticity-induced hopping
couplings += [Hopping(honeycomb, "np.sin(2 * theta_ell) * ellipticity / 2", "B_px", "B_py", (0,0))] # intra-site ellipticity-induced hopping

H = Hamiltonianbuilder(honeycomb, parameters, reciprocalcoords=["kx", "ky"])
H.set_on_site_energies(onsite)
H.add_couplings(couplings)
H.build()
eigva, eigve = H.eigh()

In [19]:
from tightbinding.plotting import plot_bands_3D
# plot_bands_3D(eigva.sel(band=[2,3,4,5]), escale = 1)

## Computing the Euler curvature

Now that we have the Hamiltonian, we can compute the Euler curvature : $\mathrm{Eu}_{n,n+1}  \!=\! \left\langle \partial_{k_x} u_{n,\bm{k}} | \partial_{k_y} u_{n+1,\bm{k}}\right\rangle \!-\! \left\langle \partial_{k_y} u_{n,\bm{k}} | \partial_{k_x} u_{n+1,\bm{k}} \right\rangle$ using the fact that the Euler curvature for a given pair of eigenvectors $\text{Eu}\left(\ket{r_{n,\bm{k}}}, \, \ket{r_{n+1,\bm{k}}}\right)$ is equal to the (Abelian) Berry curvature of a complexified band with wavefunction $\ket{s_{n,n+1,{\bm k}}} =  \left( \ket{r_{n,\bm{k}}} + i \ket{r_{n+1,\bm{k}}} \right) / \sqrt{2}$ ([ref](https://doi.org/10.1038/s41567-020-0967-9)). 

Since the Euler curvature needs a real Hamiltonian, we first have to perform a basis exchange using the matrix:

$$ V = \sqrt{
        \begin{bmatrix}
            0  & 0  & 0  & 1  &  0 & 0  \\
            0  & 0  & 0  & 0  & -1 & 0  \\
            0  & 0  & 0  & 0  & 0  & -1  \\
            1  & 0  & 0  & 0  & 0  & 0 \\
            0  & -1  & 0  & 0  & 0  & 0  \\
            0  & 0  & -1  & 0  & 0  & 0
        \end{bmatrix}}
$$

In [ ]:
from scipy.linalg import sqrtm

# creating the matrix V
v = np.zeros((6,6))
v[3,0] = v[0,3] = 1
v[1,4] = v[2,5] = v[4,1] = v[5,2] = -1
v = sqrtm(v) 

H.set_type(complex) # reset the type in case this cell is run multiple times without recreating H
H.build()
H.basis_exchange(v)
H.make_real() # discard the residual complex parts, ignore 
eigva, eigve = H.eigh()

    'data': [{'colorscale': [[0.0, '#00CC96'], [1.0, '#00CC96']],
              …